In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
import random
import shutil

In [ ]:
### preparation

In [ ]:
base_dir = "/content/drive/MyDrive/miku_project/data"
true_dir = os.path.join(base_dir, "true")
false_dir = os.path.join(base_dir, "false")

output_base = "/content/drive/MyDrive/miku_project/split_data"
train_true = os.path.join(output_base, "train", "miku")
val_true = os.path.join(output_base, "val", "miku")
train_false = os.path.join(output_base, "train", "not_miku")
val_false = os.path.join(output_base, "val", "not_miku")

for d in [train_true, val_true, train_false, val_false]:
    os.makedirs(d, exist_ok=True)

def split_and_copy(src_dir, train_dst, val_dst, split_ratio=0.8):
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
    random.shuffle(files)
    split_point = int(len(files) * split_ratio)
    train_files = files[:split_point]
    val_files = files[split_point:]

    for f in train_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(train_dst, f))
    for f in val_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(val_dst, f))

random.seed(42)

split_and_copy(true_dir, train_true, val_true)
split_and_copy(false_dir, train_false, val_false)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder("/content/drive/MyDrive/miku_project/split_data/train", transform=transform)
val_dataset = ImageFolder("/content/drive/MyDrive/miku_project/split_data/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
### make and train model

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
for epoch in range(5):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/10] Loss: {loss.item():.4f}")

In [ ]:
!mkdir /content/drive/MyDrive/miku_project/model

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/miku_project/model/resnet50_miku2.pth")

In [ ]:
from sklearn.metrics import classification_report

model.load_state_dict(torch.load("/content/drive/MyDrive/miku_project/model/resnet50_miku2.pth"))
model.eval()

all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

In [ ]:
### test for given image via URL

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import requests
from io import BytesIO

In [ ]:
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("/content/drive/MyDrive/miku_project/model/resnet50_miku2.pth", map_location=device))
model = model.to(device)
model.eval()

In [ ]:
url_input = widgets.Text(
    value='',
    placeholder='Paste URL here',
    description='Image URL:',
    layout=widgets.Layout(width='600px')
)

predict_button = widgets.Button(
    description='Predict',
    button_style='success',
    tooltip='Predict image class'
)

output = widgets.Output()

In [ ]:
def predict_image_from_url(url, model):
    try:
        response = requests.get(url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert("RGB")
        display(img)

        img_tensor = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(img_tensor)
            probs = torch.softmax(outputs, dim=1)
            pred = torch.argmax(probs, dim=1).item()
            confidence = probs[0][pred].item()

        return pred, confidence
    except Exception as e:
        print(f"Error: {e}")
        return None, None

In [ ]:
def on_predict_clicked(b):
    with output:
        clear_output()
        url = url_input.value.strip()
        pred, conf = predict_image_from_url(url, model)
        if pred is None:
            print("failed")
        else:
            label = "Miku is in a picture!" if pred == 0 else "Miku isn't in a picture."
            print(f"{label} (Confidence: {conf:.2f})")

In [ ]:
predict_button.on_click(on_predict_clicked)
display(url_input, predict_button, output)